# Trade Mid Calc

Calculates the recent average traded price from multiple market venues for a single currency pair. The recent trade prices and sizes are weighted by the time elapsed since the trade occurred

## Calc

On trade update from the venue, trade weight is calculated for the latest trade from each venue respectively. Trade Weight decays as a function of the time passed since the last trade, older the last trade lower the weight


$venueTradeWeight = min (1.0, tradeSizeRatio * \frac{tradeSize}{volumeOfInterest}) * exp ({\frac{now - latestTradeTime}{tradeWeightDecayLifeNanos}})$

After calculating recent trade weight of each each venue, a weighted average price of trade is calculated. Lets say finalTradeValue.

Again finalTradeValue is checked against price collar. If not with in price collar then it is dropped from trade mid calc

If Price collar is successful then


$resultantTradeMid = prevFinalTradeValue + (finalTradeValue - prevFinalTradeValue) * recentTradeWeight * midUpperBound$

where 

- midUpperBound is a configured val < 1, e.g: 0.86 for EURHUF, 0.94 for AUDUSD etc
- recentTradeWeight is sum of all weights from all venues



## Inputs


### Trade Mid Params

- trade venues
- mid upper bound (0.75, 0.9, 0.83) - Upper bound of the weight a most recent trade can contribute to trade mid (w * recentTradeMid + (1 - w) * emaTradeMid)  
- volume of interest (e.g: 1m)
- trade Size Ratio (e.g: 3.0) - Multiplier applied to the trade size that's normalized by volume of interest
- trade weight decay life nanos (1.0e9) - Trade weight exponential decay life in nano seconds
- min trade size (1.0e6) - Minimum trade size assumed for each trade update
- trade mid publish tolerance (1.0e-5) - publish threshold



### Market data

* Trades


### Other Calc Inputs

* Quote Mid
* Ref Price


## Calc Triggers
* Price callar update
* trade update
* quote mid


## Calc Steps

* Drop if outside price collar
* tradeVal = tradeVal + (recenttradeVal - tradeVal) * venueTradeWeight * midUpperBoundConfig (<1)
* Check whether tradeVal is bounded by bounding vwap bid/ask
* Publish final value